In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import (Reshape,LeakyReLU,Dropout,Conv2DTranspose, Add, Conv2D, MaxPool2D, Dense,
                                     Flatten, InputLayer, BatchNormalization, Input, )
from tensorflow.keras.optimizers import Adam

In [ ]:
BATCH_SIZE = 128
IM_SHAPE = (64,64,3)
LEARNING_RATE = 2e-4
LATENT_DIM=100
EPOCHS=20
IMG_PATH=''### specify path to images

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "PATH TO IMAGES", label_mode=None, image_size=(IM_SHAPE[0], IM_SHAPE[1]), batch_size=BATCH_SIZE
)

In [ ]:
dataset

In [ ]:
def preprocess(image):
    return tf.cast(image, tf.float32) / 127.5 - 1.0

In [ ]:
train_dataset = (
    dataset
    .map(preprocess)
    .unbatch()
    .shuffle(buffer_size = 1024, reshuffle_each_iteration = True)
    .batch(BATCH_SIZE,drop_remainder=True)
    .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
for d in train_dataset.take(1):
    print(d.shape)

<H1>Modeling</H1>



<h2>Our generator model:</h2>
<br>Note: 
<br> 
<ul>
    <li> kernel_size is divisible by number of strides</li>
    <li> final activation is tanh</li>
    <li> leaky relu of 0.2 used</li>
    <li> batchnormalization used</li>
   
</ul>

In [ ]:
generator=tf.keras.Sequential([
  Input(shape=(LATENT_DIM,)),
  Dense(4*4*LATENT_DIM),
  Reshape((4,4,LATENT_DIM)),

  Conv2DTranspose(512,kernel_size=4,strides=2, padding='same'),
  BatchNormalization(),
  LeakyReLU(0.2),

  Conv2DTranspose(256,kernel_size=4,strides=2, padding='same'),
  BatchNormalization(),
  LeakyReLU(0.2),

  Conv2DTranspose(128,kernel_size=4,strides=2, padding='same'),
  BatchNormalization(),
  LeakyReLU(0.2),

  Conv2DTranspose(3,kernel_size=4,strides=2, activation=tf.keras.activations.tanh, padding='same'),

],name='generator')

In [ ]:
generator.summary()

<h2>Our discriminator model:</h2>
<br>Note: 
<br> 
<ul>
    <li> kernel_size is divisible by number of strides</li>
    <li> final activation is sigmoid</li>
    <li> leaky relu of 0.2 used</li>
    <li> batchnormalization used</li>
   
</ul>

In [ ]:
discriminator=tf.keras.Sequential([
  Input(shape=(IM_SHAPE[0],IM_SHAPE[1],3)),

  Conv2D(64,kernel_size=4,strides=2, padding='same'),
  LeakyReLU(0.2),

  Conv2D(128,kernel_size=4,strides=2, padding='same'),
  BatchNormalization(),
  LeakyReLU(0.2),
  
  Conv2D(256,kernel_size=4,strides=2, padding='same'),
  BatchNormalization(),
  LeakyReLU(0.2),

  Conv2D(1,kernel_size=4,strides=2, padding='same'),

  Flatten(),
  Dense(1,activation='sigmoid')
  

],name='discriminator')

In [ ]:
discriminator.summary()

<h2>Show Image callback:</h2><br>
<br>Used to save generated images in a folder, while training is going on.
<br> 


In [ ]:
class ShowImage(tf.keras.callbacks.Callback):
    def __init__(self, latent_dim=100):
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        n=6
        k=0
        out=self.model.generator(tf.random.normal(shape=(36, self.latent_dim)))
        plt.figure(figsize=(16,16))
        for i in range(n):
            for j in range(n):
                ax=plt.subplot(n,n,k+1)
                plt.imshow((out[k]+1)/2,)
                plt.axis('off')
                k+=1
        plt.savefig("generated/gen_images_epoch_{}.png".format(epoch+1))

<h2>GAN Model (overriding train_step method):</h2>
<br>Note: Some modifications Inspired by this github repository: <a href='https://github.com/soumith/ganhacks'>Soumith Chintala's GAN Hacks</a>
<br> 
<ul>
    <li> label smoothing </li>
    <li> label flipping</li>
   
</ul>


In [ ]:
class GAN(tf.keras.Model):
    def __init__(self,discriminator,generator):
        super(GAN,self).__init__()
        self.discriminator=discriminator
        self.generator=generator

    def compile(self,d_optimizer,g_optimizer,loss_fn):
        super(GAN,self).compile()
        self.d_optimizer=d_optimizer
        self.g_optimizer=g_optimizer
        self.loss_fn=loss_fn
        self.d_loss_metric=tf.keras.metrics.Mean(name='d_loss')
        self.g_loss_metric=tf.keras.metrics.Mean(name='g_loss')
    
    @property
    def metrics(self):
        return [self.d_loss_metric,self.g_loss_metric]
  
    def train_step(self,real_images):
        batch_size=tf.shape(real_images)[0]

        ######## Discriminator
        random_noise=tf.random.normal(shape=(batch_size,LATENT_DIM))
        fake_images=self.generator(random_noise)
        
        real_labels=tf.ones((batch_size,1))
        real_labesl+=0.25*tf.random.uniform((batch_size,1),minval=-1,maxval=1)### label smoothing (check out link above)
        
        fake_labels=tf.zeros((batch_size,1))
        fake_labels+=0.25*tf.random.uniform((batch_size,1),)### label smoothing(check out link above)

        with tf.GradientTape() as recorder:
            real_predictions=self.discriminator(real_images)
            d_loss_real=self.loss_fn(real_labels,real_predictions)

            fake_predictions=self.discriminator(fake_images)
            d_loss_fake=self.loss_fn(fake_labels,fake_predictions)
            d_loss=d_loss_real+d_loss_fake
      
        partial_derivatives = recorder.gradient(d_loss,self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(zip(partial_derivatives, self.discriminator.trainable_weights))

        ############# Generator
        random_noise=tf.random.normal(shape=(batch_size,LATENT_DIM))
        flipped_fake_labels=tf.ones((batch_size,1))### label flipping

        with tf.GradientTape() as recorder:
        

            fake_predictions=self.discriminator(self.generator(random_noise))
            g_loss=self.loss_fn(flipped_fake_labels,fake_predictions)
      
        partial_derivatives = recorder.gradient(g_loss,self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(partial_derivatives, self.generator.trainable_weights))

        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
    
        return {'d_loss':self.d_loss_metric.result(),'g_loss':self.g_loss_metric.result()}

In [ ]:
gan=GAN(discriminator,generator)
gan.compile(
    d_optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE,beta_1=0.5),
    g_optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE,beta_1=0.5),
    loss_fn=tf.keras.losses.BinaryCrossentropy(),
)

In [ ]:
!mkdir generated

In [ ]:
EPOCHS=100
history=gan.fit(train_dataset,epochs=EPOCHS,callbacks=[ShowImage(LATENT_DIM)])

In [ ]:
plt.plot(history.history['d_loss'])
plt.plot(history.history['g_loss'])
plt.title('GAN Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['d_loss', 'g_loss'], loc='upper left')
plt.show()